In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 1) requirements.txt'teki tüm paketleri zorla yeniden yükle
%pip install --upgrade --force-reinstall \
    -r /content/drive/MyDrive/colab_notebooks/son_ball-action-spotting/requirements.txt \
    -f https://download.pytorch.org/whl/cu118/torch_stable.html

# 2) Bellekteki eski C-uzantılarını temizlemek için runtime’ı yeniden başlat
import os
os.kill(os.getpid(), 9)

Looking in links: https://download.pytorch.org/whl/cu118/torch_stable.html, https://download.pytorch.org/whl/cu118/torch_stable.html
  Cloning https://github.com/NVIDIA/VideoProcessingFramework (to revision v2.0.0) to /tmp/pip-req-build-7t5gi7kh
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/VideoProcessingFramework /tmp/pip-req-build-7t5gi7kh
  Running command git checkout -q 7ab13cb035fb3aa33eca0b76d2815336211945f3
  Resolved https://github.com/NVIDIA/VideoProcessingFramework to commit 7ab13cb035fb3aa33eca0b76d2815336211945f3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 1.1 MB/s eta 0

In [1]:
import sys
project_path = "/content/drive/MyDrive/colab_notebooks/son_ball-action-spotting"
sys.path.append(project_path)

In [2]:
import os
import gc

import torch
from typing import List, Tuple

import argparse
from pathlib import Path

from tqdm import tqdm
import numpy as np

from src.ball_action.annotations import raw_predictions_to_actions, prepare_game_spotting_results, save_npz
from src.utils import get_best_model_path, get_video_info
from src.predictors import MultiDimStackerPredictor
#from src.frame_fetchers import NvDecFrameFetcher
from src.frame_fetchers.opencv import OpencvFrameFetcher as NvDecFrameFetcher
from src.ball_action import constants

os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = "video_codec;h264"

RESOLUTION = "720p"
INDEX_SAVE_ZONE = 1
TTA = False


CONFIG = {
    # Drive kok dizini (tek noktadan degistirilebilir)
    "ROOT": Path("/content/drive/MyDrive"),
    # ROOT'e gore model yolu
    "MODEL_PATH": "colab_notebooks/son_ball-action-spotting/data/ball_action/experiments/sampling_weights_001/fold_1/model-029-0.871587.pth",
    # Girdi video yolu (mutlak veya goreceli)
    "VIDEO_PATH": "/content/drive/MyDrive/cropped_2023-2024_11_fraport-tav-antalyaspor_besiktas.mp4",
    # Tahmin ciktilari icin cikti dizini ROOT'e gore (action model ile ayni)
    "OUTPUT_DIR": "colab_notebooks/goal_detection/2023-2024_11_fraport-tav-antalyaspor_besiktas/ball",
    # Soccernet mac kimligi
    "GAME_NAME": "2023-2024_11_fraport-tav-antalyaspor_besiktas",
}

# Uretilen yollar
ROOT = CONFIG["ROOT"]
MODEL_PATH = ROOT / CONFIG["MODEL_PATH"]
VIDEO_PATH = Path(CONFIG["VIDEO_PATH"])
OUTPUT_DIR = ROOT / CONFIG["OUTPUT_DIR"]
GAME_NAME = CONFIG["GAME_NAME"]

# Inference ayarlari
BATCH_SIZE = 16                # her batch'te islenecek kare sayisi
INDEX_SAVE_ZONE = 1            # sinir karelerini atlama
TTA = False                    # test zamani augmentasyon bayragi

# Cikti dizinini olustur
(OUTPUT_DIR / GAME_NAME).mkdir(parents=True, exist_ok=True)

def parse_arguments():
    parser = argparse.ArgumentParser(description="Top islemleri icin inference")
    parser.add_argument("--use_saved_predictions", action="store_true",
                        help="Ham npz dosyasini yukle, yeniden inference yapma")
    parser.add_argument("--gpu_id", type=int, default=0,
                        help="CUDA cihaz ID (ornegin 0)")
    args, _ = parser.parse_known_args()    # bilinmeyen argslari ignore et
    return args


def get_raw_predictions(predictor: MultiDimStackerPredictor,
                        video_path: Path,
                        frame_count: int) -> Tuple[List[int], np.ndarray]:
    """
    Her kare batchi uzerinde modeli calistirir ve ham tahminleri dondurur.
    """
    print(f"Islenen video: {video_path}, kare sayisi: {frame_count}")

    fetcher = NvDecFrameFetcher(video_path, gpu_id=predictor.device.index)
    fetcher.num_frames = frame_count
    idx_gen = predictor.indexes_generator
    min_idx = idx_gen.clip_index(0, frame_count, INDEX_SAVE_ZONE)
    max_idx = idx_gen.clip_index(frame_count, frame_count, INDEX_SAVE_ZONE)

    frame_to_pred = {}
    predictor.reset_buffers()
    with tqdm(total=frame_count, desc="Kareler") as pbar:
        while True:
            batch_frames, batch_idxs = [], []
            for _ in range(BATCH_SIZE):
                frame = fetcher.fetch_frame()
                idx = fetcher.current_index
                if frame is None or idx >= max_idx:
                    break
                batch_frames.append(frame)
                batch_idxs.append(idx)
            if not batch_frames:
                break
            with torch.no_grad():
                preds = [predictor.predict(f, i) for f, i in zip(batch_frames, batch_idxs)]
            for pred, idx in preds:
                if idx >= min_idx and pred is not None:
                    frame_to_pred[idx] = pred.cpu().numpy()
            torch.cuda.empty_cache()
            gc.collect()
            pbar.update(len(batch_frames))
            if batch_idxs and batch_idxs[-1] >= max_idx:
                break
    predictor.reset_buffers()
    sorted_idxs = sorted(frame_to_pred)
    raw_preds = np.stack([frame_to_pred[i] for i in sorted_idxs], axis=0)
    print(f"Ham tahminler tamamlandi: {len(sorted_idxs)} kare.")
    return sorted_idxs, raw_preds

def run_inference(use_saved: bool, gpu_id: int):
    # Video bilgisi
    vid_info = get_video_info(VIDEO_PATH)
    assert abs(vid_info["fps"] - constants.video_fps) < 0.1, "FPS uyusmuyor"

    # Ham tahminler icin npz dosya yolu
    raw_npz = OUTPUT_DIR / GAME_NAME / "1_raw_predictions.npz"
    if use_saved and raw_npz.exists():
        with np.load(raw_npz) as arr:
            frame_idxs = arr["frame_indexes"]
            raw_preds = arr["raw_predictions"]
        print(f"Kaydedilmis tahminler yuklendi: {raw_npz}")
    else:
        predictor = MultiDimStackerPredictor(str(MODEL_PATH), device=f"cuda:{gpu_id}", tta=TTA)
        frame_idxs, raw_preds = get_raw_predictions(predictor, VIDEO_PATH, vid_info["frame_count"])
        save_npz(raw_npz, frame_idxs, raw_preds)

    # Class eylemlerine post-isleme uygula
    class2actions = raw_predictions_to_actions(frame_idxs, raw_preds)

    # Sonuc JSON'u hazirla
    prepare_game_spotting_results({1: class2actions}, GAME_NAME, OUTPUT_DIR)


if __name__ == "__main__":
    args = parse_arguments()
    run_inference(args.use_saved_predictions, args.gpu_id)


Islenen video: /content/drive/MyDrive/cropped_2023-2024_11_fraport-tav-antalyaspor_besiktas.mp4, kare sayisi: 12169


Kareler: 100%|█████████▉| 12153/12169 [07:11<00:00, 28.14it/s]


Ham tahminler tamamlandi: 12124 kare.
NPZ kaydedildi: /content/drive/MyDrive/colab_notebooks/goal_detection/2023-2024_11_fraport-tav-antalyaspor_besiktas/ball/2023-2024_11_fraport-tav-antalyaspor_besiktas/1_raw_predictions.npz
Predicted 182 PASS actions
Predicted 135 DRIVE actions
Ball results spotting kaydedildi: /content/drive/MyDrive/colab_notebooks/goal_detection/2023-2024_11_fraport-tav-antalyaspor_besiktas/ball/2023-2024_11_fraport-tav-antalyaspor_besiktas/ball_results_spotting.json
